# Symulator Tomografu
**Autor: Kacper Magnuszewski**

Projekt wykorzystuje on interaktywny Jupyter Notebook. Ma on przedstawić działanie tomografu komputerowego. Polega to na badaniu prześwietlaniu za pomocą promieni rentgenowskich w celu uzyskania pełnego obrazu przekroju danego obiektu. Zaprojektowany model symulatora tomografu korzysta z geometri stożkowej - przy każdym prześwietleniu badany jest przekrój obiektu 1D przy użyciu jednego emitera i wielu detektorów. Zestaw emitera i detektorów obraca się wokół obiektu, aby stworzyć sinogram, a następnie go zinterpretować i odtworzyć wejściowy obraz. Możliwe jest wybranie opcji filtrowania sinogramu, co pozwoli na ograniczenie powstałych przy rekonstrukcji zakłóceń.

### Wykorzystane narzędzia
Do stworzenia projektu wykorzystany został język programowania Python oraz wymienione poniżej biblioteki. Ich funkcje w programie to:
 - numpy - Wykorzystane do obliczeń
 - matplotlib - Drukowanie obrazu wejściowego i wyjściowego oraz sinogramu
 - cv2 - Interpretacja i przetwarzanie obrazu
 - ipywidgets - Dodanie interaktywnych suwaków i przycisków
 - IPython - Czyszczenie wyników drukowanych przez komórki
 - pydicom - Zapisanie obrazu wyjściowego w formacie dicom
 - datetime - Dodanie daty do pliku dicom

In [289]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

from ipywidgets import interact
from IPython.display import clear_output

from pydicom.dataset import Dataset, FileDataset
from datetime import datetime

Wczytanie obrazu i jego przetworzenie / przygotowanie do użycia w programie

In [290]:
IMAGE = "przykladowe_skany/Shepp_logan.jpg"

# Read, set the size and normalize the image
image = cv2.imread(IMAGE)
imageSize = max(image.shape[:2])
radius = imageSize // 2
image = cv2.resize(image, (imageSize, imageSize))
image = image / 255.0
center = [imageSize // 2, imageSize // 2] # [x, y]

### Wykorzystane funkcje

Funkcje **_emitterPosition()_** oraz **_detectorPosition()_** służą do obliczenia pozycji emitera i detektorów przy danych obrocie **_ALPHA_**.

In [291]:
def emitterPosition(center:tuple, radius:int, angle:int):
    """Returns the emitter position\n
    x - previous x value\n
    y - previous y value\n
    radius - radius of the circle (pixels) on which the emitters and detectors move\n
    angle - the angle (in degrees) by which the position of the emitter changes"""
    angleRadians = np.radians(angle)

    newX = center[0] + radius * np.cos(angleRadians)
    newY = center[0] + radius * np.sin(angleRadians)

    return (round(newX), round(newY))


def detectorPositions(center:tuple, radius:int, angle:int, detectorsRange:int, detectorsNum:int):
    """Returns a list of detectors' position\n
    emitterPosition - a tuple with x and y position of the emitter\n
    radius - radius of the circle (pixels) on which the emitters and detectors move\n
    angle - the angle (in degrees) by which the position of the emitter changes\n
    detectorsRange - the range of the detectors (in degreed) on which the detectors can be placed\n
    detectorsNum - the number of detectors used in the CT scan"""
    translation = np.radians((360 - detectorsRange) // 2)
    angleRadians = np.radians(angle)
    distBtwnDet = np.radians(detectorsRange / detectorsNum)

    detectors = []

    for i in range(detectorsNum):
        newX = center[0] + radius * np.cos(angleRadians + translation + i * distBtwnDet)
        newY = center[0] + radius * np.sin(angleRadians + translation + i * distBtwnDet)
        detectors.append((round(newX), round(newY)))

    return detectors


Funkcja **_bresenham()_** zwraca punkty, które tworzą linię biegnącą od emitera do detektora.

Funkcja **_singleScan()_** zwraca linie, biegnące od emitera do wszystkich z detektorów.

In [292]:
def bresenham(x0:int, y0:int, x1:int, y1:int):
    """Returns the positions of the pixels from a point (x0, y0) to a point (x1, y1)"""
    dx = abs(x1 - x0)
    sx = 1 if x0 < x1 else -1
    dy = abs(y1 - y0)
    sy = 1 if y0 < y1 else -1
    error = dx - dy
    line = []

    while x0 != x1 or y0 != y1:
        line.append((x0, y0))

        e2 = 2 * error
        if e2 >= -dy:
            if x0 == x1: break
            error -= dy
            x0 += sx
        if e2 <= dx:
            if y0 == y1: break
            error += dx
            y0 += sy
    
    return line


def singleScan(radius:int, detectorsRange:int, detectorsNum:int, alpha:int):
    """Perform a single scan of the image. Get the emitter and detectors' positions and return the value of the line from the emitter to each detector."""
    lines = []
    emitter = emitterPosition(center, radius, alpha)
    detectors = detectorPositions(center, radius, alpha, detectorsRange, detectorsNum)

    for i in range(len(detectors)):
        lines.append(bresenham(emitter[0], emitter[1], detectors[i][0], detectors[i][1]))

    return lines

Funkcja **_radonTransform()_** sumuje wartości liczbowe wszystkich punktów należących do danej linii, zamieszcza je w liście, którą po przetworzeniu wszystkich linii zwraca.

Funkcja **_backprojection()_** to odwrotna transformata radona, która uzupełnia obraz wyjściowy o odpowiednie wartości pobrane z sinogramu.

Funkcja **_filterSinogram()_** służy do filtracji sinogramu w celu usunięcia zakłóceń postałych przy tworzeniu obrazu wyjściowego.

In [293]:
def radonTransform(image, imageSize:int, lines:list):
    """Sums up all of the numerical values of pixels on the image that are contained in the line"""
    lineValues = []
    for line in lines:
            sum = 0
            for point in line:
                x, y = point
                if 0 <= x < imageSize and 0 <= y < imageSize:
                    sum += image[y][x].mean()
            lineValues.append(sum)
    return lineValues


def backprojection(sinogram, imageSize:int, lines:list, iteration:int):
    outputImage = np.zeros((imageSize, imageSize))

    for detector, line in enumerate(lines):
        for point in line:
            x, y = point
            if 0 <= x < imageSize and 0 <= y < imageSize:
                outputImage[y, x] += sinogram[detector, iteration]
    
    return outputImage


def filterSinogram(sinogram):
    n = sinogram.shape[0]
    filter = 2 * np.abs(np.fft.fftfreq(n, 1.0).reshape(-1, 1))
    result = np.fft.ifft(np.fft.fft(sinogram, axis = 0) * filter, axis = 0)
    result = np.clip(np.real(result), 0, 1)
    return result

Funkcja **_createPlot()_** służy do wydrukowania obrazu wejściowego, sinogramu oraz obrazu wyjściowego.

In [294]:
def createPlot(image, sinogram, outputImage):
    fig, axs = plt.subplots(1, 3)
    axs[0].imshow(image)
    axs[1].imshow(sinogram, cmap = 'gray')
    axs[2].imshow(outputImage, cmap = 'gray')

In [295]:
def saveDicom(image, fileName, imageSize):
    meta = Dataset()
    meta.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.2'
    meta.MediaStorageSOPInstanceUID = "1.2.3"
    meta.ImplementationClassUID = "1.2.3.4"
    meta.TransferSyntaxUID = '1.2.840.10008.1.2'
    
    ds = FileDataset(fileName, {}, file_meta=meta, preamble=b"\0" * 128)
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = 'MONOCHROME2'
    ds.Rows, ds.Columns = imageSize, imageSize
    ds.BitsAllocated = 16
    ds.BitsStored = 16
    ds.PixelRepresentation = 0
    
    image = (image * (2**16 - 1)).astype(np.uint16)
    ds.PixelData = image.tobytes()
    
    ds.PatientName = "Anonimowy"
    ds.PatientID = "151746"
    ds.PatientBirthDate = "20021203"
    ds.PatientSex = "M"
    ds.StudyDate = datetime.now().strftime('%Y%m%d')
    ds.StudyTime = datetime.now().strftime('%H%M%S')
    ds.Modality = "CT"
    ds.Manufacturer = "Kacper Magnuszewski"
    
    ds.save_as(fileName, write_like_original=False)

Finkcja **_ctScan()_** wykonuje pełen skan i przetwarza obraz wejściowy stosując podane parametry obrotu emitera i detektorów w każdej iteracji oraz drukuje obraz wyjściowy zgodnie z podanymi kryteriami (czy obraz ma być przefiltrowany oraz czy program ma pokazać każdą z iteracji).

In [296]:
def ctScan(image, imageSize, alpha, radius, detectorsNum, detectorsRange, showIterations, filter):
    iterations = round(360 // alpha)
    sinogram = np.zeros((detectorsNum, iterations), dtype = np.float32)
    outputImage = np.zeros((imageSize, imageSize), dtype = np.float32)

    for i in range(iterations):
        lines = singleScan(radius, detectorsRange, detectorsNum, i * alpha)
        sinogram[:, i] = radonTransform(image, imageSize, lines)
        outputImage += backprojection(filterSinogram(sinogram) if filter else sinogram, imageSize, lines, i)

        if showIterations or i == iterations - 1:
            clear_output(wait = True)
            createPlot(image, sinogram, outputImage)
            plt.show()

    outputImage = (outputImage - np.min(outputImage)) / (np.max(outputImage) - np.min(outputImage))

    saveDicom(outputImage, "test.dcm", imageSize)

### Porównanie obrazów filtrowanych i bez filtracji

Obrazy filtrowane są ostrzejsze, wyraźniejsze i dużo bardziej przypominają oryginał. Na obrazie bez filtracji widać wiele artefaktów, które sprawiają, że jest on rozmazany. Większość fragmentów badanego obiektu jest widoczna, jednak szczegóły są ciężkie do rozpoznania.

---

Obraz niefiltrowany:

![title](NoFilter.png)

Obraz przefiltrowany:

![title](Filtered.png)

---

Aby uruchomić program należy wykonać wszystkie komórki, wybrać opcje suwakami oraz okienkami "_checkbox_", które pojawią się na samym dole dokumentu po czym kliknąć pole **_START_**.

In [297]:
@interact(ALPHA = (0.5, 4, 0.25), DETECTORS_NUM = (90, 720, 90), RANGE = (45, 270, 45), SHOW_ITERATIONS = False, FILTER = False, START = False)
def main(ALPHA = 2, DETECTORS_NUM = 180, RANGE = 180, SHOW_ITERATIONS = False, FILTER = False, START = False):
    if START:
        ctScan(image, imageSize, ALPHA, radius, DETECTORS_NUM, RANGE, SHOW_ITERATIONS, FILTER)
        START = False

interactive(children=(FloatSlider(value=2.0, description='ALPHA', max=4.0, min=0.5, step=0.25), IntSlider(valu…